# Linear Regressions

In [1]:
import pandas as pd
import logging
import dateutil
from dateutil import parser
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point
%matplotlib inline
pd.set_option('display.max_columns', 500)

In [2]:
#import dataset of project times from "data" folder
file = "../../../data/cleaned/all_quarters__one_record_per_project.csv"
output="../../Output/Entire Sample/"
df = pd.read_csv(file)

# Initial Data Cleaning

In [3]:
#First, filter out those projects that are exclusively non-residential (defined as those without units)
df = df[df['units'] > 0]

In [4]:
df.shape

(3081, 27)

In [5]:
# drop those that don't have a firstfiled date.
df=df[pd.notnull(df['firstfiled'])]
df=df[df['firstfiled'] !='']

In [6]:
df.shape

(2793, 27)

In [7]:
#drop BP duplicates for now—this suggests that they are likely duplicates. 
#This includes records with BP "MULTIPLE" because these are mostly mega projects and we don't want them in the sample
df['bp_duplicates']=df.duplicated('dbi_permit', keep=False)
df=df[(df['bp_duplicates']==False) | (pd.isnull(df['dbi_permit']))]

In [8]:
df.shape

(2475, 28)

In [9]:
# make sure there are no duplicate first dates on same address-APN (aka drop those that have duplicate projects on same Address-APN combination)
df= df[~ df.duplicated(['address', 'apn', 'first_date'], keep=False)]

In [10]:
df.shape

(2475, 28)

In [11]:
df['project_time_years']=df['project_duration_days']/365

In [12]:
#Manual Data Cleaning for some fishy values based on top 5 shortest list above
#df.loc[(df['address']=='55 05TH ST') & (new_df['apn']=='3705039'), 'units'] = 8 #looked up on PIM. Simple adding 8 units.
#df.loc[(df['address']=='545 POST ST') & (new_df['apn']=='0306022'), 'units'] = 4 #looked up on PIM. Simple adding 4 units
#df.loc[(df['address']=='555 POST ST') & (new_df['apn']=='0306020'), 'units'] = 17 #looked up on PIM. Simple adding 4 units
#df.loc[(df['address']=='515 JOHN MUIR DR') & (new_df['apn']=='7282005'), 'units'] = 2 #looked up on PIM. Change of use to residential. 2 additional units

#based on the above cases, unit count seems to be the unit count at the site at the end of 
#the project rather than the unit count of the project itself. Because of this, I will be adjust
#unit counts that are more than the net units added to the net units added amount. If units net
#is missing, just accept unit count as true for now.

def unit_change(value):
    return_val=value['units']
    if pd.notnull(value['unitsnet']):
        if value['unitsnet']<0:
            return_val = value['units']
        elif value['unitsnet']==0:
            return_val = value['units']
        elif value['unitsnet']<value['units']:
            return_val = value['unitsnet']
        else:
            return_val=value['units']
    return return_val

df['units']=df.apply(unit_change, axis=1)
 

# Merge in Geographic Information and Make Big Dataframe

In [13]:
#first, convert points to geodataframe
crs = {'init' :'epsg:4326'}
geometry = [Point(xy) for xy in zip(df.x, df.y)]
devs = GeoDataFrame(df, crs=crs, geometry=geometry)
devs = devs.to_crs({'init': 'epsg:4326'}) 

In [14]:
#import neighborhoods
neighborhoods = gpd.read_file('../../../data/gis/41_neighborhoods/41_neighborhoods.shp')

In [15]:
#convert boundaries to geographic coordinate system to conform to points
neighborhoods = neighborhoods.to_crs({'init': 'epsg:4326'}) 

In [16]:
#First, spatial join between points and neighborhood boundaries. Set 'how' to 'left' to preserve all developments
df = gpd.sjoin(devs, neighborhoods, how = 'inner', op='within')
df.shape

(2474, 32)

In [17]:
#Create Big Projects Dataframe
df_big=df[df['units']>=10]
df_big['project_time_years'].describe()

count    137.000000
mean       6.255254
std        3.453810
min        0.750685
25%        3.854795
50%        5.945205
75%        8.273973
max       24.128767
Name: project_time_years, dtype: float64

In [18]:
#in order to use as much data as possible, I am not keeping all data and just creating time vars where we can (i.e. where they are nonnull)
def permit_time(value):
    returnval=np.nan
    if pd.notnull(value['BP_date']) & pd.notnull(value['first_date']):
        returnval=((dateutil.parser.parse(value['BP_date']) - dateutil.parser.parse(value['first_date'])).days)/365
    return returnval
def bp_time(value):
    returnval=np.nan
    if pd.notnull(value['con_date']) & pd.notnull(value['BP_date']):
        returnval=((dateutil.parser.parse(value['con_date']) - dateutil.parser.parse(value['BP_date'])).days)/365
    return returnval
def con_time(value):
    returnval=np.nan
    if pd.notnull(value['comp_date']) & pd.notnull(value['con_date']):
        returnval=((dateutil.parser.parse(value['comp_date']) - dateutil.parser.parse(value['con_date'])).days)/365
    return returnval
    
df['permit_time']=df.apply(permit_time, axis=1)
df['bp_time']=df.apply(bp_time, axis=1)
df['con_time']=df.apply(con_time, axis=1)

In [19]:
#Final sample with at least one nonmissing piece of relevant information. leaving other observations in because they don't affect graphs anyway
df[(pd.isnull(df['con_time'])) & (pd.isnull(df['permit_time'])) & (pd.isnull(df['bp_time'])) & (pd.isnull(df['project_time_years']))].shape

(1147, 35)

In [20]:
import statsmodels.formula.api as smf
import numpy as np

In [21]:
df.head()

,Unnamed: 0,BP_date,address,apn,best_date,best_stat,comp_date,con_date,dbi_permit,dropped_out,first_date,first_project_record_date,firstfiled,latest_project_record_date,latest_project_status,project_dates,project_duration_days,project_statuses,report_quarter,report_year,status,units,unitsnet,x,y,zoning,zoning_simplified,bp_duplicates,project_time_years,geometry,index_right,nhood,permit_time,bp_time,con_time
685,0,NaN,758 JAMESTOWN AV,4974008,2016-12-28,BP FILED,NaN,NaN,201612286113,False,2016-12-28,2016-12-28,2016-12-28,2016-12-28,BP FILED,"('2016-12-28', '2016-12-28', '2016-12-28')",NaN,"('BP FILED', 'BP FILED', 'BP FILED')",2,2017,Proposed,3.0,3.0,-122.389442,37.715885,NaN,NaN,False,NaN,POINT (-122.3894424438 37.7158851624),0,Bayview Hunters Point,NaN,NaN,NaN
308,0,NaN,750 JAMESTOWN AV,4976019,2009-04-16,CONSTRUCTION,01/01/2016,2009-04-16,200609071583,False,2006-09-07,2009-04-16,2006-09-07,2009-04-16,CONSTRUCTION,"('2009-04-16', '2009-04-16', '2009-04-16', '20...",3403.0,"('CONSTRUCTION', 'CONSTRUCTION', 'CONSTRUCTION...",4,2015,Under Construction,1.0,1.0,-122.389168,37.715620,RH-1,RH-1,False,9.323288,POINT (-122.389168389 37.7156197596),0,Bayview Hunters Point,NaN,NaN,6.715068
2210,0,2008-02-08,1060 GILMAN AV,4937014,2010-11-04,BP REINSTATED,NaN,NaN,200610245911,True,2006-10-24,2008-02-08,2006-10-24,2010-11-04,BP REINSTATED,"('2008-02-08', '2008-02-08', '2008-02-08', '20...",NaN,"('BP ISSUED', 'BP ISSUED', 'BP ISSUED', 'BP IS...",1,2016,Building Permit Approved,1.0,1.0,-122.389150,37.719109,RH-1,RH-1,False,NaN,POINT (-122.38915 37.719108699),0,Bayview Hunters Point,1.293151,NaN,NaN
1159,0,NaN,1212 EGBERT AV,4909003,2013-12-31,BP FILED,NaN,NaN,201312315375,False,2013-12-31,2013-12-31,2013-12-31,2013-12-31,BP FILED,"('2013-12-31', '2013-12-31', '2013-12-31', '20...",NaN,"('BP FILED', 'BP FILED', 'BP FILED', 'BP FILED...",2,2017,Proposed,2.0,2.0,-122.389008,37.720917,RH-1,RH-1,False,NaN,POINT (-122.3890075684 37.720916748),0,Bayview Hunters Point,NaN,NaN,NaN
1126,0,NaN,4101 03RD ST,5260001,2014-05-14,PL FILED,NaN,NaN,NaN,False,2014-05-14,2014-05-14,2014-05-14,2014-05-14,PL FILED,"('2014-05-14', '2014-05-14', '2014-05-14', '20...",NaN,"('PL FILED', 'PL FILED', 'PL FILED', 'PL FILED...",2,2017,Proposed,32.0,32.0,-122.388649,37.739895,NaN,NaN,True,NaN,POINT (-122.3886489868 37.73989486690001),0,Bayview Hunters Point,NaN,NaN,NaN


In [23]:
mod = smf.ols(formula='project_time_years ~ units', data=df)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:     project_time_years   R-squared:                       0.037
Model:                            OLS   Adj. R-squared:                  0.035
Method:                 Least Squares   F-statistic:                     27.45
Date:                Sat, 09 Dec 2017   Prob (F-statistic):           2.12e-07
Time:                        22:12:35   Log-Likelihood:                -1813.7
No. Observations:                 721   AIC:                             3631.
Df Residuals:                     719   BIC:                             3641.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept      4.2339      0.118     36.018      0.0